In [2]:
import requests
import zipfile
import io
import pandas as pd
import ujson
from datetime import datetime
import numpy as np

def get_fielders(x):
    try:
        return ",".join([fielder['name'] for fielder in x['wickets'][0]['fielders']])
    except:
        return ""

def extra(x,y):
    if x == 'wides' or x == 'noballs' or x == '':
        return y
    elif x == 'byes, noballs' or x == 'legbyes, noballs':
        return 1
    elif x == 'penalty, wides':
        return y-5
    else:
        return 0

def extract_match_info(data):
    info = data.get('info', {})
    date = info.get('dates', ['Unknown'])[0]
    return {
        'date': date,
        'match_number':info.get('match_type_number','Unknown'),
        'match_type':info.get('match_type','Unknown'),
        'balls_per_over': info.get('balls_per_over','Unknown'),
        'overs':info.get('overs','Unknown'),
        'venue': info.get('venue', 'Unknown'),
        'city':info.get('city','Unknown'),
        'season':info.get('season',''),
        'team_type':info.get('team_type',''),
        'teams': info.get('teams', []),
        'event_name': info.get('event', {}).get('name', 'Unknown'),
        'event_match_no': info.get('event', {}).get('match_number', 'Unknown'),
        'stage': info.get('event', {}).get('stage', 'Unknown'),
        'gender': info.get('gender', 'Unknown'),
        'outcome_winner': info.get('outcome', {}).get('winner', 'Unknown'),
        'outcome_by': info.get('outcome', {}).get('by', {}),
        'superover_win': info.get('outcome', {}).get('eliminator', 'NA'),
        'match_result': info.get('outcome', {}).get('result', 'NA'),
        'method':info.get('outcome', {}).get('method', 'NA'),
        'player_of_match': info.get('player_of_match', ['Unknown'])[0],
        'toss_winner': info.get('toss', {}).get('winner', 'Unknown'),
        'toss_decision': info.get('toss', {}).get('decision', 'Unknown'),
        'date_obj': datetime.strptime(date, '%Y-%m-%d'),
    }

def process_json_data(json_data, matchid):
    match_info = extract_match_info(json_data)
    all_data = []

    innings_number = 0
    res = ""
    for key, value in match_info["outcome_by"].items():
        res = str(value) + " " + key

    for innings in json_data.get('innings', []):
        team_name = innings['team']
        innings_number += 1
        positions = {}
        position = 1
        wickets = 0
        team_runs = 0
        team_balls = 0

        for over in innings.get('overs', []):
            valid_ball_number = 1
            for delivery in over['deliveries']:
                next_batter = None
                ball_faced = 0
                valid_ball = 0
                is_wide = 'wides' in delivery.get('extras', {})
                is_noball = 'noballs' in delivery.get('extras', {})
                if not is_wide:
                    ball_faced += 1
                if not (is_wide or is_noball):
                    ball_number = valid_ball_number
                    valid_ball_number += 1
                    valid_ball = 1
                else:
                    ball_number = valid_ball_number
                    valid_ball = 0

                if delivery.get('wickets') and delivery['wickets'][0].get('kind') not in ['retired hurt', 'retired not out']:
                    wickets += 1

                if delivery["batter"] not in positions:
                    positions[delivery['batter']] = position
                    position += 1
                    if wickets >= 1 and over['over'] + 0.1 * ball_number > 0.1:
                        next_batter = delivery['batter']               

                if delivery["non_striker"] not in positions:
                    positions[delivery['non_striker']] = position
                    position += 1
                    if wickets >= 1 and over['over'] + 0.1 * ball_number > 0.1:
                        next_batter = delivery['non_striker']

                team_runs += delivery.get('runs', {}).get('total', 0)
                team_balls += valid_ball

                all_data.append({
                    'match_id': matchid,
                    'date': match_info['date'],
                    'match_type': match_info['match_type'],
                    'event_name': match_info['event_name'],
                    'innings': innings_number,
                    'match_id_inn': matchid+'-'+str(innings_number),
                    'batting_team': team_name,
                    'bowling_team': next((team for team in match_info['teams'] if team != team_name), 'Unknown'),
                    'over': over['over'],
                    'ball': ball_number,
                    'ball_no': over['over'] + 0.1 * ball_number,
                    'batter': delivery["batter"],
                    'bat_pos': positions[delivery["batter"]],
                    'runs_batter': delivery.get('runs', {}).get('batter', 0),
                    'balls_faced': ball_faced,
                    'bowler': delivery["bowler"],
                    'valid_ball': valid_ball,
                    'runs_extras': delivery.get('runs', {}).get('extras', 0),
                    'runs_total': delivery.get('runs', {}).get('total', 0),
                    'runs_bowler': delivery.get('runs', {}).get('batter', 0) + extra(', '.join(delivery.get('extras', {}).keys()), delivery.get('runs', {}).get('extras', 0)),
                    'runs_not_boundary': delivery.get('runs', {}).get('non_boundary', False),
                    'extra_type': ', '.join(delivery.get('extras', {}).keys()),
                    'non_striker': delivery["non_striker"],
                    'non_striker_pos': positions[delivery['non_striker']],
                    'wicket_kind': delivery['wickets'][0].get('kind') if delivery.get('wickets') else None,
                    'player_out': delivery['wickets'][0].get('player_out') if delivery.get('wickets') else None, 
                    'fielders': get_fielders(delivery),
                    'runs_target': innings.get("target", {}).get("runs"),
                    'review_batter': delivery.get('review', {}).get('batter', ""),
                    'team_reviewed': delivery.get('review', {}).get('by', ""),
                    'review_decision': delivery.get('review', {}).get('decision', ""),
                    'umpire': delivery.get('review', {}).get('umpire', ""),
                    'umpires_call': delivery.get('review', {}).get('umpires_call', False),
                    'player_of_match': match_info['player_of_match'],
                    'match_won_by': match_info["outcome_winner"],
                    'win_outcome': res,
                    'toss_winner': match_info['toss_winner'],
                    'toss_decision': match_info['toss_decision'],
                    'venue': match_info['venue'],
                    'city': match_info['city'],
                    'day': match_info['date_obj'].day,
                    'month': match_info['date_obj'].month,
                    'year': match_info['date_obj'].year,
                    'season': match_info['season'],
                    'gender': match_info['gender'],
                    'team_type': match_info['team_type'],
                    'superover_winner': match_info['superover_win'],
                    'result_type': match_info['match_result'],
                    'method': match_info['method'],
                    'balls_per_over': match_info['balls_per_over'],
                    'overs': match_info['overs'],
                    'event_match_no': match_info['event_match_no'],
                    'stage': match_info['stage'],
                    'match_number': match_info['match_number'],
                    'team_runs': team_runs,
                    'team_balls': team_balls,
                    'team_wicket': wickets,
                    'new_batter': next_batter,
                })
    return all_data

In [4]:
def main():
    url = 'https://cricsheet.org/downloads/odis_male_json.zip'
    
    response = requests.get(url)
    response.raise_for_status() 
    
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        data_batches = []
        count = 0
        print('Zip file Loaded')
        for filename in z.namelist():
            if filename.endswith('.json'):
                count += 1
                print(f"match:{count}")
                with z.open(filename) as file:
                    json_data = ujson.load(file)
                    match_id = filename.split("/")[-1].split(".")[0]
                    processed_data = process_json_data(json_data, match_id)
                    data_batches.append(processed_data)
    
    combined_data = [item for sublist in data_batches for item in sublist]
    df = pd.DataFrame(combined_data).sort_values(by=["date", "match_id", "innings", "over", "ball"])
    df['batter_runs'] = df.groupby(['match_id', 'innings', 'batter'])['runs_batter'].cumsum()
    df['batter_balls'] = df.groupby(['match_id', 'innings', 'batter'])['balls_faced'].cumsum()
    df['bowler_wicket'] = (~df['wicket_kind'].isin([None, 'obstructing the field', 'retired hurt', 'run out','retired not out'])).astype(int)
    df['batting_partners'] = df.apply(lambda row: tuple(sorted([row['batter'], row['non_striker']])), axis=1)
    df['next_batter'] = df['new_batter'].shift(-1)
    
    return df

df = main()
df

Zip file Loaded
match:1
match:2
match:3
match:4
match:5
match:6
match:7
match:8
match:9
match:10
match:11
match:12
match:13
match:14
match:15
match:16
match:17
match:18
match:19
match:20
match:21
match:22
match:23
match:24
match:25
match:26
match:27
match:28
match:29
match:30
match:31
match:32
match:33
match:34
match:35
match:36
match:37
match:38
match:39
match:40
match:41
match:42
match:43
match:44
match:45
match:46
match:47
match:48
match:49
match:50
match:51
match:52
match:53
match:54
match:55
match:56
match:57
match:58
match:59
match:60
match:61
match:62
match:63
match:64
match:65
match:66
match:67
match:68
match:69
match:70
match:71
match:72
match:73
match:74
match:75
match:76
match:77
match:78
match:79
match:80
match:81
match:82
match:83
match:84
match:85
match:86
match:87
match:88
match:89
match:90
match:91
match:92
match:93
match:94
match:95
match:96
match:97
match:98
match:99
match:100
match:101
match:102
match:103
match:104
match:105
match:106
match:107
match:108
match:109
ma

,match_id,date,match_type,event_name,innings,match_id_inn,batting_team,bowling_team,over,ball,...,match_number,team_runs,team_balls,team_wicket,new_batter,batter_runs,batter_balls,bowler_wicket,batting_partners,next_batter
1085343,64814,2002-12-29,ODI,India tour of New Zealand,1,64814-1,New Zealand,India,0,1,...,1927,1,1,0,None,0,1,0,"(NJ Astle, SP Fleming)",None
1085344,64814,2002-12-29,ODI,India tour of New Zealand,1,64814-1,New Zealand,India,0,2,...,1927,1,2,0,None,0,1,0,"(NJ Astle, SP Fleming)",None
1085345,64814,2002-12-29,ODI,India tour of New Zealand,1,64814-1,New Zealand,India,0,3,...,1927,1,3,0,None,0,2,0,"(NJ Astle, SP Fleming)",None
1085346,64814,2002-12-29,ODI,India tour of New Zealand,1,64814-1,New Zealand,India,0,4,...,1927,2,4,0,None,1,3,0,"(NJ Astle, SP Fleming)",None
1085347,64814,2002-12-29,ODI,India tour of New Zealand,1,64814-1,New Zealand,India,0,5,...,1927,2,5,0,None,0,2,0,"(NJ Astle, SP Fleming)",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315386,1442992,2024-08-07,ODI,India tour of Sri Lanka,2,1442992-2,India,Sri Lanka,25,3,...,4754,138,153,8,None,30,22,0,"(Kuldeep Yadav, Washington Sundar)",None
1315387,1442992,2024-08-07,ODI,India tour of Sri Lanka,2,1442992-2,India,Sri Lanka,25,4,...,4754,138,154,8,None,30,23,0,"(Kuldeep Yadav, Washington Sundar)",None
1315388,1442992,2024-08-07,ODI,India tour of Sri Lanka,2,1442992-2,India,Sri Lanka,25,5,...,4754,138,155,8,None,30,24,0,"(Kuldeep Yadav, Washington Sundar)",None
1315389,1442992,2024-08-07,ODI,India tour of Sri Lanka,2,1442992-2,India,Sri Lanka,25,6,...,4754,138,156,9,None,30,25,1,"(Kuldeep Yadav, Washington Sundar)",Mohammed Siraj


<h1>Partnership Analysis</h1>

In [5]:
import pandas as pd

# Group by and calculate partnership runs and balls
x = df.groupby(['match_id_inn', 'date','batting_team', 'bowling_team', 'batting_partners'])[['runs_total', 'balls_faced']].sum().reset_index().rename(columns={'runs_total' :  'partnership_runs','balls_faced' :  'partnership_balls'})


y = df.groupby(['match_id_inn', 'date','batting_team', 'bowling_team', 'batting_partners', 'batter'])[['runs_batter', 'balls_faced']].sum().reset_index()

# Merge the two dataframes
z = x.merge(y, on=['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batting_partners'], how='left')

# Handle the case where only one batter contributed to the partnership
one_batter_contributions = z.groupby(['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batting_partners']).filter(lambda g :   len(g)   ==   1)
one_batter_contributions['missing_batter'] = one_batter_contributions.apply(lambda row :   [b for b in row['batting_partners'] if b.strip() != row['batter']][0], axis=1)

# Create missing batter rows with 0 runs and balls faced
missing_batter = one_batter_contributions.copy()
missing_batter['batter'] = missing_batter['missing_batter']
missing_batter['runs_batter'] = 0
missing_batter['balls_faced'] = 0
missing_batter = missing_batter.drop(columns=['missing_batter'])

# Combine all batter contributions
combined = pd.concat([z, missing_batter], ignore_index=True)

# Split batters into batter_1 and batter_2
combined['batter_1'], combined['batter_2'] = zip(*combined['batting_partners'])

# Assign runs and balls faced to each batter
combined['batter_1_runs'] = combined.apply(lambda row :   row['runs_batter'] if row['batter']   ==   row['batter_1'] else 0, axis=1)
combined['batter_1_balls'] = combined.apply(lambda row :   row['balls_faced'] if row['batter']   ==   row['batter_1'] else 0, axis=1)
combined['batter_2_runs'] = combined.apply(lambda row :   row['runs_batter'] if row['batter']   ==   row['batter_2'] else 0, axis=1)
combined['batter_2_balls'] = combined.apply(lambda row :   row['balls_faced'] if row['batter']   ==   row['batter_2'] else 0, axis=1)

# Aggregate the final partnership data
final = combined.groupby(['match_id_inn', 'date','batting_team', 'bowling_team', 'batting_partners', 'partnership_runs', 'partnership_balls', 'batter_1', 'batter_2']).agg({
    'batter_1_runs' :  'sum', 
    'batter_1_balls' :  'sum', 
    'batter_2_runs' :  'sum', 
    'batter_2_balls' :  'sum'
}).reset_index()

# Calculate strike rates and percentage contribution
final['batter_1_SR'] = round(final['batter_1_runs'] * 100 / final['batter_1_balls'], 2)
final['batter_2_SR'] = round(final['batter_2_runs'] * 100 / final['batter_2_balls'], 2)
final['batter_1_%'] = round(final['batter_1_runs'] * 100 / final['partnership_runs'], 2)
final['batter_2_%'] = round(final['batter_2_runs'] * 100 / final['partnership_runs'], 2)

# Add partnership wickets information
partnership_wickets = df.groupby(['match_id_inn', 'date','batting_team', 'bowling_team', 'batting_partners']).agg({'team_wicket' :   'min'}).reset_index().assign(team_wicket=lambda x :   x['team_wicket'] + 1)
final = final.merge(partnership_wickets, on=['match_id_inn', 'date','batting_team', 'bowling_team', 'batting_partners'], how='left')
final = final.rename(columns={'team_wicket' :   'partnership_wicket'})

# Add partnership start runs and end runs
partnership_runs = df.groupby(['match_id_inn', 'date','batting_team', 'bowling_team', 'batting_partners']).agg({
    'team_runs' :   ['first', 'last'],
    'team_wicket' :  ['first','last']
}).reset_index()
partnership_runs.columns = ['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batting_partners', 'partnership_start_runs', 'partnership_end_runs','partnership_start_wkts','partnership_end_wkts']
final = final.merge(partnership_runs, on=['match_id_inn', 'date','batting_team', 'bowling_team', 'batting_partners'], how='left')

# Determine if the partnership is unbeaten
final['unbeaten'] = final['partnership_start_wkts']   ==   final['partnership_end_wkts']

# Select final columns
final = final[['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batting_partners', 
               'partnership_runs', 'partnership_balls', 'partnership_wicket', 
               'batter_1', 'batter_1_runs', 'batter_1_balls', 'batter_1_SR', 'batter_1_%',
               'batter_2', 'batter_2_runs', 'batter_2_balls', 'batter_2_SR', 'batter_2_%',
               'partnership_start_runs', 'partnership_start_wkts', 'partnership_end_runs', 'partnership_end_wkts','unbeaten']]

final

,match_id_inn,date,batting_team,bowling_team,batting_partners,partnership_runs,partnership_balls,partnership_wicket,batter_1,batter_1_runs,...,batter_2,batter_2_runs,batter_2_balls,batter_2_SR,batter_2_%,partnership_start_runs,partnership_start_wkts,partnership_end_runs,partnership_end_wkts,unbeaten
0,1000887-1,2017-01-13,Australia,Pakistan,"(B Stanlake, MS Wade)",33,26,10,B Stanlake,1,...,MS Wade,31,20,155.00,93.94,235,9,268,9,True
1,1000887-1,2017-01-13,Australia,Pakistan,"(CA Lynn, TM Head)",39,28,3,CA Lynn,16,...,TM Head,21,16,131.25,53.85,13,2,52,3,False
2,1000887-1,2017-01-13,Australia,Pakistan,"(DA Warner, TM Head)",13,27,1,DA Warner,7,...,TM Head,5,9,55.56,38.46,0,0,13,1,False
3,1000887-1,2017-01-13,Australia,Pakistan,"(GJ Maxwell, MR Marsh)",11,20,5,GJ Maxwell,9,...,MR Marsh,2,9,22.22,18.18,68,4,78,5,False
4,1000887-1,2017-01-13,Australia,Pakistan,"(GJ Maxwell, MS Wade)",82,88,6,GJ Maxwell,51,...,MS Wade,28,43,65.12,34.15,79,5,160,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38798,997995-1,2016-08-16,United Arab Emirates,Scotland,"(Rameez Shahzad, Shaiman Anwar)",43,25,5,Rameez Shahzad,16,...,Shaiman Anwar,25,12,208.33,58.14,144,4,187,5,False
38799,997995-2,2016-08-16,Scotland,United Arab Emirates,"(CD Wallace, KJ Coetzer)",15,13,1,CD Wallace,6,...,KJ Coetzer,9,6,150.00,60.00,4,0,15,1,False
38800,997995-2,2016-08-16,Scotland,United Arab Emirates,"(CS MacLeod, KJ Coetzer)",28,40,2,CS MacLeod,12,...,KJ Coetzer,16,22,72.73,57.14,19,1,43,2,False
38801,997995-2,2016-08-16,Scotland,United Arab Emirates,"(CS MacLeod, PL Mommsen)",165,214,3,CS MacLeod,91,...,PL Mommsen,74,110,67.27,44.85,43,2,208,3,False


In [16]:
partnerships = final.groupby(['batting_partners','batter_1','batter_2']).agg({
    'partnership_runs' :   ['sum', 'max'],
    'partnership_balls' :   'sum',
    'batter_1_runs' :   'sum',
    'batter_1_balls' :   'sum',
    'batter_2_runs' :   'sum',
    'batter_2_balls' :   'sum',
    'unbeaten' :   'sum',
    'match_id_inn' :   'nunique'
}).reset_index()

# Flattening the MultiIndex columns
partnerships.columns = ['batting_partners', 'batter_1', 'batter_2',
                        'partnership_runs', 'Highest', 'partnership_balls',
                        'batter_1_runs', 'batter_1_balls', 'batter_2_runs', 
                        'batter_2_balls', 'notouts', 'innings']

# Calculating additional columns
partnerships['batter_1_SR'] = round(partnerships['batter_1_runs'] * 100 / partnerships['batter_1_balls'], 2)
partnerships['batter_2_SR'] = round(partnerships['batter_2_runs'] * 100 / partnerships['batter_2_balls'], 2)
partnerships['batter_1_%'] = round(partnerships['batter_1_runs'] * 100 / partnerships['partnership_runs'], 2)
partnerships['batter_2_%'] = round(partnerships['batter_2_runs'] * 100 / partnerships['partnership_runs'], 2)
partnerships['sr_diff'] = abs(partnerships['batter_1_SR'] - partnerships['batter_2_SR'])

# Selecting and sorting the final columns
partnerships = partnerships[['batting_partners', 'innings', 'notouts', 'partnership_runs', 'partnership_balls', 
                             'Highest', 'batter_1', 'batter_1_runs', 'batter_1_balls', 
                             'batter_1_%', 'batter_1_SR', 'batter_2', 'batter_2_runs', 
                             'batter_2_balls', 'batter_2_%', 'batter_2_SR', 'sr_diff']]

# Filtering and sorting the results
result = partnerships[partnerships['partnership_runs'] >= 1000].sort_values(by='sr_diff', ascending=False).head(30)

result

,batting_partners,innings,notouts,partnership_runs,partnership_balls,Highest,batter_1,batter_1_runs,batter_1_balls,batter_1_%,batter_1_SR,batter_2,batter_2_runs,batter_2_balls,batter_2_%,batter_2_SR,sr_diff
5739,"(R Dravid, V Sehwag)",39,2,2203,3259,268,R Dravid,690,1703,31.32,40.52,V Sehwag,1404,1556,63.73,90.23,49.71
5497,"(NT Paranavitana, TM Dilshan)",39,4,1509,2342,207,NT Paranavitana,566,1340,37.51,42.24,TM Dilshan,811,1002,53.74,80.94,38.70
2785,"(G Gambhir, V Sehwag)",77,3,3814,5033,233,G Gambhir,1486,2654,38.96,55.99,V Sehwag,2142,2379,56.16,90.04,34.05
1796,"(CJL Rogers, DA Warner)",41,1,2053,3154,200,CJL Rogers,805,1711,39.21,47.05,DA Warner,1144,1443,55.72,79.28,32.23
2121,"(DA Warner, EJM Cowan)",30,0,1291,2102,214,DA Warner,761,1060,58.95,71.79,EJM Cowan,441,1042,34.16,42.32,29.47
1040,"(Azhar Ali, Mohammad Hafeez)",36,4,1554,2853,287,Azhar Ali,561,1431,36.10,39.20,Mohammad Hafeez,943,1422,60.68,66.32,27.12
1333,"(BJ Watling, LRPL Taylor)",21,4,1089,1805,253,BJ Watling,416,931,38.20,44.68,LRPL Taylor,619,874,56.84,70.82,26.14
989,"(Asad Shafiq, Sarfraz Ahmed)",28,2,1275,1875,173,Asad Shafiq,523,980,41.02,53.37,Sarfraz Ahmed,710,895,55.69,79.33,25.96
4638,"(M Vijay, S Dhawan)",41,1,1748,2796,289,M Vijay,698,1465,39.93,47.65,S Dhawan,975,1331,55.78,73.25,25.60
1732,"(CH Gayle, D Ganga)",25,2,1051,1907,162,CH Gayle,604,947,57.47,63.78,D Ganga,368,960,35.01,38.33,25.45


In [6]:
final['max']=abs(final['batter_1_%']-final['batter_2_%'])
final['min']=np.minimum(final['batter_1_runs'], final['batter_2_runs'])
final[(final['min']  ==  0)].sort_values(by=['partnership_runs','max'],ascending=False).head(30)

,match_id_inn,date,batting_team,bowling_team,batting_partners,partnership_runs,partnership_balls,partnership_wicket,batter_1,batter_1_runs,...,batter_2_balls,batter_2_SR,batter_2_%,partnership_start_runs,partnership_start_wkts,partnership_end_runs,partnership_end_wkts,unbeaten,max,min
3891,1144170-1,2019-03-13,Sri Lanka,South Africa,"(CAK Rajitha, I Udana)",58,34,10,CAK Rajitha,0,...,25,220.00,94.83,133,9,189,10,False,94.83,0
288,1020013-2,2017-01-30,Australia,New Zealand,"(JR Hazlewood, MP Stoinis)",54,24,10,JR Hazlewood,0,...,24,200.00,88.89,226,9,280,10,False,88.89,0
28947,578619-1,2013-06-11,West Indies,India,"(DJG Sammy, KAJ Roach)",51,27,10,DJG Sammy,51,...,8,0.00,0.00,182,9,233,9,True,100.00,0
35596,736441-1,2014-08-21,Zimbabwe,South Africa,"(E Chigumbura, T Panyangara)",46,34,10,E Chigumbura,42,...,12,0.00,0.00,119,9,165,10,False,91.30,0
34660,667729-2,2014-09-05,India,England,"(RA Jadeja, UT Yadav)",44,24,10,RA Jadeja,43,...,2,0.00,0.00,209,9,253,10,False,97.73,0
8765,1288315-1,2022-04-02,Australia,Pakistan,"(A Zampa, SA Abbott)",44,36,10,A Zampa,0,...,28,150.00,95.45,167,9,210,10,False,95.45,0
4318,1144511-2,2019-06-22,West Indies,New Zealand,"(CR Brathwaite, O Thomas)",41,24,10,CR Brathwaite,41,...,4,0.00,0.00,249,9,286,10,False,100.00,0
25308,455235-2,2010-06-21,Bangladesh,Pakistan,"(Imrul Kayes, Tamim Iqbal)",39,47,1,Imrul Kayes,0,...,27,125.93,87.18,0,0,39,1,False,87.18,0
12371,1373571-1,2023-09-07,South Africa,Australia,"(L Ngidi, T Bavuma)",37,34,10,L Ngidi,0,...,24,145.83,94.59,185,9,222,10,False,94.59,0
13808,1384423-2,2023-11-01,New Zealand,South Africa,"(GD Phillips, MJ Henry)",34,31,10,GD Phillips,34,...,9,0.00,0.00,133,9,167,10,False,100.00,0


<h1>Entry Point Analysis</h1>

In [11]:
# Adjust team runs and balls for the striker's first appearance
striker_first = df.groupby(['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batter']).agg({
    'team_runs' :   'first', 
    'team_balls' :   'first',
    'team_wicket' :   'first',
    'runs_total' :   'first', 
    'valid_ball' :   'first',
    'bat_pos'  :   'first'
}).reset_index()

# Adjust team runs and balls by subtracting the batter's contributions
striker_first['adjusted_team_runs'] = striker_first['team_runs'] - striker_first['runs_total']
striker_first['adjusted_team_balls'] = striker_first['team_balls'] - striker_first['valid_ball']


# Rename columns for merging consistency
striker_first = striker_first.rename(columns={'adjusted_team_runs' :   'entry_runs', 
                                              'adjusted_team_balls' :   'entry_balls',
                                              'team_wicket' :  'entry_wickets'})

# Find the first occurrence of a batter as a non-striker without adjustments
non_striker_first = df.groupby(['match_id_inn', 'date', 'batting_team', 'bowling_team', 'non_striker']).agg({
    'team_runs' :   'first', 
    'team_balls' :   'first',
    'runs_total' :   'first', 
    'valid_ball' :   'first',
    'team_wicket' :   'first',
    'non_striker_pos'  :   'first'
}).reset_index().rename(columns={'non_striker' :   'batter'})

non_striker_first['adjusted_team_runs'] = non_striker_first['team_runs'] - non_striker_first['runs_total']
non_striker_first['adjusted_team_balls'] = non_striker_first['team_balls'] - non_striker_first['valid_ball']

non_striker_first = non_striker_first.rename(columns={'adjusted_team_runs' :   'entry_runs', 
                                              'adjusted_team_balls' :   'entry_balls',
                                              'team_wicket' :  'entry_wickets',
                                              'non_striker_pos' :  'bat_pos'})
entry_point_first = pd.concat([striker_first, non_striker_first]).sort_values(by=['match_id_inn', 'batter', 'entry_balls']).drop_duplicates(subset=['match_id_inn', 'batter'], keep='first').reset_index(drop=True)

striker_last = df.groupby(['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batter']).agg({
    'team_runs' :   'last', 
    'team_balls' :   'last', 
    'team_wicket' :   'last'
}).reset_index()



# Rename columns for merging consistency
striker_last = striker_last.rename(columns={'team_runs' :   'exit_runs', 
                                            'team_balls' :   'exit_balls',
                                            'team_wicket' :   'exit_wickets'})

# Find the first occurrence of a batter as a non-striker without adjustments
non_striker_last = df.groupby(['match_id_inn', 'date', 'batting_team', 'bowling_team', 'non_striker']).agg({
    'team_runs' :   'last', 
    'team_balls' :   'last',
    'team_wicket' :   'last'
}).reset_index().rename(columns={'non_striker' :   'batter'})


non_striker_last = non_striker_last.rename(columns={'team_runs' :   'exit_runs', 
                                              'team_balls' :   'exit_balls',
                                                   'team_wicket' :   'exit_wickets'})

entry_point_last = pd.concat([striker_last, non_striker_last]).sort_values(by=['match_id_inn', 'batter', 'exit_balls',],ascending=False).drop_duplicates(subset=['match_id_inn', 'batter'], keep='first').reset_index(drop=True)

w=entry_point_first.merge(entry_point_last,on=['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batter'])
x=df.groupby(['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batter'])[['runs_batter','balls_faced']].sum()
w=w.merge(x,on=['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batter'])
w['fifties']=(w['runs_batter']>=50)&(w['runs_batter']<=99)
w['hundreds']=w['runs_batter']>=100
w['team_runs']=w['exit_runs']-w['entry_runs']
w['wkts']=w['exit_wickets']-w['entry_wickets']
w=w[['match_id_inn', 'date', 'batting_team', 'bowling_team', 'batter','entry_runs','entry_balls','entry_wickets','exit_runs','exit_balls','exit_wickets','team_runs','wkts','runs_batter','balls_faced','bat_pos','fifties','hundreds']]
w    

,match_id_inn,date,batting_team,bowling_team,batter,entry_runs,entry_balls,entry_wickets,exit_runs,exit_balls,exit_wickets,team_runs,wkts,runs_batter,balls_faced,bat_pos,fifties,hundreds
0,1000887-1,2017-01-13,Australia,Pakistan,B Stanlake,235,274,9,268,300,9,33,0,1,6,11,False,False
1,1000887-1,2017-01-13,Australia,Pakistan,CA Lynn,13,28,2,52,56,3,39,1,16,12,4,False,False
2,1000887-1,2017-01-13,Australia,Pakistan,DA Warner,0,0,0,13,27,1,13,1,7,18,1,False,False
3,1000887-1,2017-01-13,Australia,Pakistan,GJ Maxwell,67,78,4,160,186,6,93,2,60,56,6,True,False
4,1000887-1,2017-01-13,Australia,Pakistan,JP Faulkner,160,186,6,170,204,7,10,1,5,12,8,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43182,997995-2,2016-08-16,Scotland,United Arab Emirates,CD Wallace,0,0,0,15,13,1,15,1,6,7,2,False,False
43183,997995-2,2016-08-16,Scotland,United Arab Emirates,CS MacLeod,15,13,1,208,267,3,193,2,103,122,3,False,True
43184,997995-2,2016-08-16,Scotland,United Arab Emirates,KJ Coetzer,0,0,0,43,53,2,43,2,25,28,1,False,False
43185,997995-2,2016-08-16,Scotland,United Arab Emirates,PL Mommsen,43,53,2,229,286,3,186,1,80,116,4,True,False


In [158]:
z=w[(w['entry_balls']<90)&(w['bat_pos']>=5)].groupby('batter').agg({'match_id_inn' :  'nunique',
                                                                  'bat_pos' :  'median',
                                                                  'entry_runs' :  'mean',
                                                                  'entry_balls' :  'mean',
                                                                  'exit_runs' :  'mean',
                                                                  'exit_balls' :  'mean',
                                                                  'team_runs' :  'mean',
                                                                  'runs_batter' :  ['sum','mean'],
                                                                  'fifties' :  'sum',
                                                                  'hundreds' :  'sum'}).reset_index()
z.sort_values(by=[('match_id_inn','nunique'),('runs_batter','mean')],ascending=False).head(50)

batter match_id_inn bat_pos entry_runs entry_balls  \
                            nunique  median       mean        mean   
527    Shakib Al Hasan           49     5.0  35.979592   53.897959   
15          AD Mathews           44     5.0  49.704545   62.340909   
357           MS Dhoni           36     6.0  44.527778   59.527778   
585       Yuvraj Singh           34     5.0  51.029412   67.088235   
492           SK Raina           33     5.0  46.303030   62.939394   
483        SC Williams           32     5.0  39.625000   64.062500   
530       Shoaib Malik           31     5.0  35.419355   58.322581   
165         EJG Morgan           29     5.0  47.103448   62.448276   
361        Mahmudullah           28     6.0  38.392857   58.785714   
289         KJ O'Brien           27     5.0  38.740741   63.888889   
7            A Symonds           24     5.0  47.916667   59.875000   
531      Sikandar Raza           24     5.0  38.875000   60.416667   
561         TWM Latham           24     5.0  49.000000   66.375000   
372      Mohammad Nabi           24     6.0  46.625000   67.125000   
288       KIC Asalanka           22     5.0  41.090909   51.772727   
127          DA Miller           22     6.0  56.954545   69.636364   
241         JC Buttler           22     5.5  42.409091   56.363636   
541            T Taibu           22     5.0  30.772727   60.000000   
138           DJ Bravo           22     5.0  38.818182   60.818182   
412     PD Collingwood           21     5.0  42.523810   63.190476   
367      Misbah-ul-Haq           21     5.0  38.095238   57.000000   
164       E Chigumbura           21     6.0  39.285714   66.047619   
565         Umar Akmal           20     5.0  35.450000   60.400000   
125           D Ramdin           20     5.0  46.900000   61.850000   
381    Mushfiqur Rahim           20     5.5  44.450000   60.900000   
335         MEK Hussey           18     5.0  39.500000   57.500000   
481          SB Styris           18     5.0  45.222222   57.777778   
265          JP Duminy           18     5.0  53.611111   71.055556   
3           A Flintoff           17     5.0  41.294118   58.352941   
63           BA Stokes           16     5.0  55.500000   57.250000   
281         KA Pollard           16     6.0  42.562500   64.875000   
232     Inzamam-ul-Haq           16     5.0  43.562500   63.000000   
315          LPC Silva           16     5.0  43.187500   59.750000   
185         GD Elliott           16     5.0  48.562500   62.062500   
11      AB de Villiers           15     5.0  43.666667   59.066667   
199          H Klaasen           15     5.0  48.533333   55.400000   
387           N Pooran           15     5.0  42.666667   54.533333   
471     S Matsikenyeri           15     5.0  29.800000   53.866667   
426           R Dravid           14     5.0  49.142857   59.428571   
97         CD McMillan           14     5.0  43.571429   59.285714   
466              S Bau           14     5.0  46.357143   62.928571   
368  Mohammad Ashraful           14     5.0  34.928571   58.642857   
189         GJ Maxwell           13     5.0  44.076923   58.769231   
552         TM Dilshan           13     6.0  49.461538   76.076923   
353         MP Stoinis           13     5.0  41.769231   54.846154   
34            AT Carey           12     6.0  47.000000   58.750000   
153   DPMD Jayawardene           12     5.0  40.250000   58.833333   
437      RD Berrington           12     5.0  48.166667   57.666667   
264          JO Holder           12     7.0  47.250000   66.333333   
359         MV Boucher           12     6.0  50.333333   69.000000   

      exit_runs  exit_balls   team_runs runs_batter            fifties  \
           mean        mean        mean         sum       mean     sum   
527  112.612245  154.244898   76.632653        1887  38.510204      14   
15   132.659091  164.772727   82.954545        1637  37.204545      13   
357  135.305556  176.388889   90.777778        1496  41.555556       8   
585  